In [1]:
import pandas as pd
from scipy import stats
import os
import glob
import numpy as np

In [2]:
path = r'G:\My Drive\Mahad\Light Level Experiments\Data Output from Analysis\MainVideoProcessingCodes_v4\Step6_v4\\'
outpath = r'./dataFolders/Output/Step6_v4/'

In [3]:
def in_ranges(x,bins):
    return [((x>=y[0])&(x<=y[1])) for y in bins]

In [4]:
notoutliers = pd.read_csv(path + 'AllLight_EveryMoth_notOutliers.csv')
notoutliers['DiscoveryTime'] = notoutliers.DiscoveryTime.div(100,axis = 'index')
notoutliers = notoutliers.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'])
notoutliers.head()

,In_Frame,Out_Frame,Proboscis,DiscoveryTime,trialNum,name,zscore,outlier_ID,condition
0,45940,53025,46222.0,2.82,0,L0.1_c-3_m20,0.251222,False,Low
1,55058,60805,55341.0,2.83,1,L0.1_c-3_m20,0.249691,False,Low
2,67612,69165,67744.0,1.32,2,L0.1_c-3_m20,0.480929,False,Low
3,72461,74177,72606.0,1.45,3,L0.1_c-3_m20,0.461021,False,Low
4,77368,79850,77499.0,1.31,4,L0.1_c-3_m20,0.482460,False,Low


In [5]:
Bins = [(0, 10), (5, 15), (10, 20), (15, 25), (20, 30)]
label = ['(0,10)', '(5,15)', '(10,20)', '(15,25)', '(20,30)']

In [6]:
binned = notoutliers['trialNum'].apply(lambda x: pd.Series(in_ranges(x,Bins), label))

In [7]:
binned

,"(0,10)","(5,15)","(10,20)","(15,25)","(20,30)"
0,True,False,False,False,False
1,True,False,False,False,False
2,True,False,False,False,False
3,True,False,False,False,False
4,True,False,False,False,False
...,...,...,...,...,...
426,True,False,False,False,False
427,True,False,False,False,False
428,True,False,False,False,False
429,True,False,False,False,False


In [8]:
notoutliers = notoutliers.join(binned)

In [9]:
test = notoutliers.name.str.split('_', expand = True)
notoutliers['lightLevel'] = test[0]

In [10]:
notoutliers.head()

,In_Frame,Out_Frame,Proboscis,DiscoveryTime,trialNum,name,zscore,outlier_ID,condition,"(0,10)","(5,15)","(10,20)","(15,25)","(20,30)",lightLevel
0,45940,53025,46222.0,2.82,0,L0.1_c-3_m20,0.251222,False,Low,True,False,False,False,False,L0.1
1,55058,60805,55341.0,2.83,1,L0.1_c-3_m20,0.249691,False,Low,True,False,False,False,False,L0.1
2,67612,69165,67744.0,1.32,2,L0.1_c-3_m20,0.480929,False,Low,True,False,False,False,False,L0.1
3,72461,74177,72606.0,1.45,3,L0.1_c-3_m20,0.461021,False,Low,True,False,False,False,False,L0.1
4,77368,79850,77499.0,1.31,4,L0.1_c-3_m20,0.482460,False,Low,True,False,False,False,False,L0.1


In [11]:
notoutliers.columns

Index(['In_Frame', 'Out_Frame', 'Proboscis', 'DiscoveryTime', 'trialNum',
       'name', 'zscore', 'outlier_ID', 'condition', '(0,10)', '(5,15)',
       '(10,20)', '(15,25)', '(20,30)', 'lightLevel'],
      dtype='object')

In [12]:
for bin_label in label:
    sub_df = notoutliers[notoutliers[bin_label] == True]
    for l, df in sub_df.groupby('lightLevel'):
        n = len(df)
        print('%s and %s bin has %i visits' %(l, bin_label, n))        

L0.1 and (0,10) bin has 152 visits
L50 and (0,10) bin has 95 visits
L0.1 and (5,15) bin has 116 visits
L50 and (5,15) bin has 69 visits
L0.1 and (10,20) bin has 74 visits
L50 and (10,20) bin has 37 visits
L0.1 and (15,25) bin has 53 visits
L50 and (15,25) bin has 18 visits
L0.1 and (20,30) bin has 44 visits
L50 and (20,30) bin has 8 visits


### mark early and late visits

In [13]:
notoutliers.loc[notoutliers.lightLevel == 'L50', 'trialNum'].max()

24

In [14]:
Bins = [(0, 4), (14,24)]
label = ['early', 'late']

In [15]:
earlyLate_binned = notoutliers['trialNum'].apply(lambda x: pd.Series(in_ranges(x,Bins), label))

In [16]:
test = notoutliers.join(earlyLate_binned)
test.head()

,In_Frame,Out_Frame,Proboscis,DiscoveryTime,trialNum,name,zscore,outlier_ID,condition,"(0,10)","(5,15)","(10,20)","(15,25)","(20,30)",lightLevel,early,late
0,45940,53025,46222.0,2.82,0,L0.1_c-3_m20,0.251222,False,Low,True,False,False,False,False,L0.1,True,False
1,55058,60805,55341.0,2.83,1,L0.1_c-3_m20,0.249691,False,Low,True,False,False,False,False,L0.1,True,False
2,67612,69165,67744.0,1.32,2,L0.1_c-3_m20,0.480929,False,Low,True,False,False,False,False,L0.1,True,False
3,72461,74177,72606.0,1.45,3,L0.1_c-3_m20,0.461021,False,Low,True,False,False,False,False,L0.1,True,False
4,77368,79850,77499.0,1.31,4,L0.1_c-3_m20,0.482460,False,Low,True,False,False,False,False,L0.1,True,False


In [17]:
for l, df in test.groupby('lightLevel'):
    n = len(df[df.early == True])
    print('%s has %s early visits' %(l, n))
    n = len(df[df.late == True])
    print('%s has %s early visits' %(l, n))

L0.1 has 75 early visits
L0.1 has 57 early visits
L50 has 47 early visits
L50 has 22 early visits


## get bins for first and last n visits

In [18]:
high_mend = test.loc[test.lightLevel == 'L50', 'trialNum'].max()
high_mstrt = test.loc[test.lightLevel == 'L50', 'trialNum'].max()-10

low_mend = test.loc[test.lightLevel == 'L0.1', 'trialNum'].max()
low_mstrt = test.loc[test.lightLevel == 'L0.1', 'trialNum'].max()-10

In [19]:
high_mend, high_mstrt

(24, 14)

In [20]:
low_mend, low_mstrt

(48, 38)

In [ ]:
Bins = [(0, 2)]
label = ['early3']

early3 = test['trialNum'].apply(lambda x: pd.Series(in_ranges(x,Bins), label))
test = test.join(early3)

In [ ]:
Bins = [(high_mstrt, high_mend)]
label = ['late10_high']

Late10_binned_high = test.loc[test.lightLevel == 'L50', 'trialNum'].apply(lambda x: pd.Series(in_ranges(x,Bins), label))

In [ ]:
Bins = [(low_mstrt, low_mend)]
label = ['late10_low']

Late10_binned_low = test.loc[test.lightLevel == 'L0.1', 'trialNum'].apply(lambda x: pd.Series(in_ranges(x,Bins), label))

In [ ]:
new_test = test.join(Late10_binned_high, how='outer')
final_test = pd.concat([new_test, Late10_binned_low], axis=1, sort=False)

In [ ]:
final_test.head()

In [ ]:
sub = final_test.loc[final_test.lightLevel == 'L0.1', :]
len(sub[sub['late10_low'] == True])

In [ ]:
sub = final_test.loc[final_test.lightLevel == 'L50', :]
len(sub[sub['late10_high'] == True])

In [ ]:
sub = final_test.loc[final_test.lightLevel == 'L0.1', :]
len(sub[sub['early3'] == True])

In [ ]:
sub = final_test.loc[final_test.lightLevel == 'L50', :]
len(sub[sub['early3'] == True])

In [ ]:
final_test.to_csv(outpath + 'AllLight_EveryMoth_notOutliers_withBins.csv')

## generate and store the pde and data for plotting later

In [ ]:
from scipy.stats import gaussian_kde

In [ ]:
bin_center = np.linspace(0,40,100)
delta = np.diff(bin_center)[0]

In [ ]:
notoutliers = pd.read_csv(outpath + 'AllLight_EveryMoth_notOutliers_withBins.csv')

### store the sliding window pde

In [ ]:
label = ['(0,10)', '(5,15)', '(10,20)', '(15,25)', '(20,30)']

SampleSize = pd.DataFrame(columns = ['L0.1', 'L50'], index = label)
pde_df = pd.DataFrame(columns = label, index = bin_center)

for l, subdf in notoutliers.groupby('lightLevel'):
    for bin_label in label:
        df = subdf[subdf[bin_label] == True]
        data = df.DiscoveryTime  
        SampleSize.loc[bin_label,l] = len(data)
        
        kde = gaussian_kde(data)
        temp = kde.pdf(bin_center)
        temp /= delta * np.sum(temp)
        pde_df.loc[:,bin_label]=temp
    pde_df.to_csv(outpath+ l + '_pde.csv')
SampleSize.to_csv(outpath+ 'samplesize.csv')

### Store the pde for early and late visits

In [ ]:
label = ['early', 'late']

SampleSize = pd.DataFrame(columns = ['L0.1', 'L50'], index = label)
pde_df = pd.DataFrame(columns = label, index = bin_center)

for l, subdf in notoutliers.groupby('lightLevel'):
    for bin_label in label:
        df = subdf[subdf[bin_label] == True]
        data = df.DiscoveryTime  
        SampleSize.loc[bin_label,l] = len(data)
        
        kde = gaussian_kde(data)
        temp = kde.pdf(bin_center)
        temp /= delta * np.sum(temp)
        pde_df.loc[:,bin_label]=temp
    pde_df.to_csv(outpath+ l + '_earlyLate_pde.csv')
SampleSize.to_csv(outpath+ 'earlyLateSamplesize.csv')

## store the pde for first 3 and last 10 visits

In [ ]:
superlabel = ['early3', 'last10']
SampleSize = pd.DataFrame(columns = ['L0.1', 'L50'], index = superlabel)
pde_df = pd.DataFrame(columns = superlabel, index = bin_center)

for l, subdf in notoutliers.groupby('lightLevel'):
    if l == 'L0.1':
        label = ['early3', 'late10_low']
    else:
        label = ['early3', 'late10_high']
    
    for bin_label, bl in zip(label, superlabel):
        df = subdf[subdf[bin_label] == True]
        data = df.DiscoveryTime  
        SampleSize.loc[bl,l] = len(data)
        
        kde = gaussian_kde(data)
        temp = kde.pdf(bin_center)
        temp /= delta * np.sum(temp)
        pde_df.loc[:,bl]=temp
    pde_df.to_csv(outpath+ l + '_firstlast_pde.csv')
SampleSize.to_csv(outpath+ 'firstlastSamplesize.csv')